# Loading CAO points information into a pandas data frame, using CAO website

http://www.cao.ie/index.php?page=points&p=2021

***

## INTRODUCTION

## WHAT IS CAO

### LOADING THE DATA

In [1]:
import requests as rq
import re

In [2]:
resp = rq.get ("http://www2.cao.ie/points/l8.php")
print (resp)

<Response [200]>


REGULAR EXPRESSIONS

In [7]:
re_course = re.compile(r"([A-Z]{2}[0-9]{3})(.*)([0-9]{3})(\*?) *")

LOOP THROUGH THE LINES

In [11]:
no_lines = 0

# Loop
for line in resp.iter_lines():
    # Match only lines with courses
    if re_course.fullmatch(line.decode("iso 8859-1")):
        # Add one to the lines counter
        no_lines = no_lines + 1
        #print (line)
        # Pick out the relevant lines
        csv_version = re_course.sub(r"\1,\2,\3,\4", line.decode("iso 8859-1"))
        # Print the CSV
        print (csv_version)
        
print ("Total number of lines:", no_lines)

AL830,  General Nursing                                      451*     ,444,
AL832,  Mental Health Nursing                                440*     ,431,
AL841,  Athletic and Rehabilitation Therapy                  477      ,476,
AL854,  Sport Management (with international placement)      300      ,299,
AL870,  Applied Psychology                                   484*     ,467,
CW138,  Sport and Exercise Science                           430      ,423,
CW438,  Construction (options)                               260      ,251,
CW468,  Architectural Technology                             291      ,283,
CW548,  Mechanical Engineering                               330      ,328,
CW568,  Aerospace Engineering                                371      ,359,
CW758,  Professional Social Care Practice                    360      ,358,
PC405,  Social  Political and Community Studies              263      ,235,
PC410,  Arts and Humanities (fulfils Teaching Council requ   262      ,230,
PC411,  Engl

#### Regular expressions

### CAO DATA 2019, 2020, 2021

### DATA COMPARISON

# CONCLUSION

***

## REFERENCES

## LINKS TO CHECK